# DEC Supervised Clustering

This notebook focuses on using some kind of supervision for helping the clustering. This includes different techniques that we discussed with Dr. Ece Kamar from Microsoft during her visit to UMN. The different techniques include: 
    1. Initializing cluster centers based on highly confident galaxy images
        a. Run K-Means on encoded features learned from autoencoder for top 10-20 high confident images of each class and check whether K-Means is able to successfully cluster the features into two different cluster with high purity
        b. Analyze the PCA plot of the encoded features learned from autoencoder for top 10-20 high confident images of each class for visible distinct cluster
    2. Providing some kind of feedback on how good/bad clustering was in the previous iteration
        a. Not sure?

Some other techniques that we are also trying to include and analyse are:
    3. Clustering over a quadrant of the image to try to get rid of the worm shape PCA plot
        a. Input images are crops of the original images but of the top left quadrant
    4. Using different clustering techniques which include denisty and heirarchical clustering
        a. DBSCAN
        b. Agglomerative Clustering
        c. Heirarchical Clustering
        d. Spectral Clustering
        
        refer to the webpage: https://scikit-learn.org/stable/modules/clustering.html